# Attention
Divide nuclei into low or high attention.  
On each half, do a rollup to patch quartiles.  
Test whether RF does better on high vs low attention data.

This notebook was part-way implemented when we changed course.

In [1]:
import datetime
import numpy as np
import pandas as pd
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
THISCLASS=4  # testing
CLASSPATH='/home/jrm/Adjeroh/Naved/CP_80K/Output4/'
NUCFILENAME='Process100_Nucleus.csv'
PATCHFILENAME='Process100_Image.csv'
ATTFILENAME='attention_avg_temp_per_nucleus.csv'
CNNFILENAME='/home/jrm/Adjeroh/Naved/CP_80K/CNN/scores.csv'

## Load CNN scores
Retain filenames of patches predicted correctly by the CNN

In [3]:
scores=pd.read_csv(CNNFILENAME)  # same file for all classes
scores

,patchname,ground_truth,predicted_label,softmax_0,softmax_1,softmax_2,softmax_3,softmax_4,softmax_5
0,TCGA-02-0004-01Z-00-DX1_4800_3600,0,0,9.999977e-01,2.466497e-08,1.495720e-07,7.797728e-08,6.044772e-08,1.899641e-06
1,TCGA-02-0004-01Z-00-DX1_4800_4200,0,0,9.999999e-01,4.827287e-13,4.781109e-11,1.163105e-14,1.071110e-11,8.704473e-08
2,TCGA-02-0004-01Z-00-DX1_4500_4200,0,0,1.000000e+00,1.984801e-10,8.819246e-09,1.731268e-09,6.153791e-11,1.787250e-09
3,TCGA-02-0004-01Z-00-DX1_5400_3300,0,0,9.999882e-01,3.108451e-07,1.531888e-06,3.748598e-06,1.214340e-06,5.000012e-06
4,TCGA-02-0004-01Z-00-DX1_5400_3900,0,0,9.999981e-01,1.063941e-14,7.136163e-12,6.937229e-16,2.445667e-11,1.867779e-06
...,...,...,...,...,...,...,...,...,...
82965,TCGA-S9-A7J1-01Z-00-DX1_72600_26400,1,2,3.380104e-07,2.480321e-02,8.634496e-01,6.795154e-07,2.734306e-03,1.090119e-01
82966,TCGA-S9-A7J1-01Z-00-DX1_69600_58500,1,4,6.624015e-08,1.844222e-03,1.204252e-01,1.170510e-05,5.556598e-01,3.220590e-01
82967,TCGA-S9-A7J1-01Z-00-DX1_70800_48300,1,4,1.643103e-07,1.568889e-03,3.065287e-01,2.030722e-05,5.148395e-01,1.770425e-01
82968,TCGA-S9-A7J1-01Z-00-DX1_74400_60600,1,4,3.034546e-06,1.155498e-04,1.003913e-01,3.831314e-05,8.207427e-01,7.870915e-02


In [4]:
print('before',scores.shape)
scores.drop(columns=['softmax_0','softmax_1','softmax_2'],inplace=True)
scores.drop(columns=['softmax_3','softmax_4','softmax_5'],inplace=True)
scores.drop(scores[scores.ground_truth!=THISCLASS].index,inplace=True)
scores.drop(scores[scores.predicted_label!=THISCLASS].index,inplace=True)
print('after',scores.shape)
scores.set_index('patchname',inplace=True)
scores

before (82970, 9)
after (785, 3)


,ground_truth,predicted_label
patchname,,
TCGA-DB-A4XF-01Z-00-DX1_1200_23700,4,4
TCGA-DB-A4XF-01Z-00-DX1_5400_22200,4,4
TCGA-DB-A4XF-01Z-00-DX1_9300_16200,4,4
TCGA-DB-A4XF-01Z-00-DX1_15900_28200,4,4
TCGA-DB-A4XF-01Z-00-DX1_46200_24900,4,4
...,...,...
TCGA-S9-A6WL-01Z-00-DX1_48000_28500,4,4
TCGA-S9-A6WL-01Z-00-DX1_51900_21900,4,4
TCGA-S9-A6WL-01Z-00-DX1_49200_28500,4,4


## Load patchnames
Retain mapping between patch filename and cell profiler image number.

In [5]:
patch_map0 = pd.read_csv(CLASSPATH+PATCHFILENAME)

In [6]:
patch_map1 = patch_map0[ ['ImageNumber','FileName_Tumor'] ]
patch_map2 = patch_map1.rename(
    columns={'ImageNumber':'PatchNumber','FileName_Tumor':'patchname'})
patch_map2['patchname'] = patch_map2['patchname'].str[:-4]
patch_map2.set_index('patchname',inplace=True)
patch_map2

,PatchNumber
patchname,
TCGA-DB-A4XF-01Z-00-DX1_10200_20700,1
TCGA-DB-A4XF-01Z-00-DX1_10200_28500,2
TCGA-DB-A4XF-01Z-00-DX1_10200_37500,3
TCGA-DB-A4XF-01Z-00-DX1_10500_27900,4
TCGA-DB-A4XF-01Z-00-DX1_10500_39300,5
...,...
TCGA-S9-A6WL-01Z-00-DX1_9900_35400,3193
TCGA-S9-A6WL-01Z-00-DX1_9900_35700,3194
TCGA-S9-A6WL-01Z-00-DX1_9900_36000,3195


In [7]:
cnn_correct_patches=patch_map2.join(scores,how='inner')
cnn_correct_patches

,PatchNumber,ground_truth,predicted_label
patchname,,,
TCGA-DB-A4XF-01Z-00-DX1_1200_23700,20,4,4
TCGA-DB-A4XF-01Z-00-DX1_15900_28200,42,4,4
TCGA-DB-A4XF-01Z-00-DX1_2100_19500,75,4,4
TCGA-DB-A4XF-01Z-00-DX1_2100_22200,76,4,4
TCGA-DB-A4XF-01Z-00-DX1_41400_63600,219,4,4
...,...,...,...
TCGA-S9-A6WL-01Z-00-DX1_53100_24900,3159,4,4
TCGA-S9-A6WL-01Z-00-DX1_9300_33000,3175,4,4
TCGA-S9-A6WL-01Z-00-DX1_9600_33900,3184,4,4


In [8]:
patch_map0=None
patch_map1=None
patch_map2=None
scores=None

## Load labels
The labels are the temperatures of the CNN attention heatmap,
specifically an average across the bounding box of each nucleus.

In [ ]:
att_df=pd.read_csv(ATTFILEPATH+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
att_df.rename(columns=cols,inplace=True)
att_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
att_df.reset_index(drop=True,inplace=True)

In [ ]:
# Later, try the RandomForestRegressor. For now use bins and classification.
bins = [0, 0.3, 0.45, 0.6, 0.75, 1.0]
labels = ['to30%', 'to45%', 'to60%', 'to75%', 'to100%']
att_df['bin'] = pd.cut(att_df['AvgTemp'], bins=bins, labels=labels)
att_df

In [ ]:
ytrain = att_df['bin'] 
ytrain.value_counts()

## Load features
These are features of nuclei as determined by our "Process100" pipeline for CellProfiler.
These are the nuclei used to set the bounding boxes for the temperature lables.
Other than that, the features and temperatures were each derived independently.

In [ ]:
nuc_df=pd.read_csv(NUCFILEPATH+NUCFILENAME)
cols={'ImageNumber':'PatchNumber'}
nuc_df.rename(columns=cols,inplace=True)
nuc_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
nuc_df.reset_index(drop=True,inplace=True)

In [ ]:
Xtrain = nuc_df.drop(columns=['PatchNumber','ObjectNumber'])
Xtrain

In [ ]:
att_df=None
nuc_df=None
print('Count Nan before:',Xtrain.isna().sum().sum())
Xtrain.fillna(0,inplace=True)
print('Count Nan after:',Xtrain.isna().sum().sum())

## Measure agreement (with a random forest classifier)
Can the RF predict the temperature of each nucleus given only the CellProfiler features?

In [ ]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

In [ ]:
print(datetime.datetime.now())
print('Cross validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('Accuracy mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

## Measure feature importance

In [ ]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf1 = None
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
rf2.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf2.important_features()
pd.set_option('display.max_rows', None)
top.loc[:200]